<a href="https://colab.research.google.com/github/KimJisanER/Dacon/blob/main/Dacon_Metabolism_0911.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 !pip install -q condacolab
import condacolab
condacolab.install()

import sys
sys.path

import condacolab
condacolab.check()

!mamba install -c conda-forge rdkit

✨🍰✨ Everything looks OK!
✨🍰✨ Everything looks OK!

                  __    __    __    __
                 /  \  /  \  /  \  /  \
                /    \/    \/    \/    \
███████████████/  /██/  /██/  /██/  /████████████████████████
              /  / \   / \   / \   / \  \____
             /  /   \_/   \_/   \_/   \    o \__,
            / _/                       \_____/  `
            |/
        ███╗   ███╗ █████╗ ███╗   ███╗██████╗  █████╗
        ████╗ ████║██╔══██╗████╗ ████║██╔══██╗██╔══██╗
        ██╔████╔██║███████║██╔████╔██║██████╔╝███████║
        ██║╚██╔╝██║██╔══██║██║╚██╔╝██║██╔══██╗██╔══██║
        ██║ ╚═╝ ██║██║  ██║██║ ╚═╝ ██║██████╔╝██║  ██║
        ╚═╝     ╚═╝╚═╝  ╚═╝╚═╝     ╚═╝╚═════╝ ╚═╝  ╚═╝

        mamba (1.4.1) supported by @QuantStack

        GitHub:  https://github.com/mamba-org/mamba
        Twitter: https://twitter.com/QuantStack

█████████████████████████████████████████████████████████████


Looking for: ['rdkit']

[+] 0.0s
[+] 0.1s
conda-forge/linux-64 

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install mordred

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.8/128.8 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 56.4 MB/s eta 0:00:00
  Created wheel for mordred: filename=mordred-1.2.0-py3-none-any.whl size=176723 sha256=641bbc2cc11986a7cd6f85fa0631685f27f407a1686ade0c3556f09466b2f1a6
  Stored in directory: /root/.cache/pip/wheels/a7/4f/b8/d4c6591f6ac944aaced7865b349477695f662388ad958743c7
Successfully built mordred


In [4]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from rdkit import Chem
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
import rdkit
from rdkit.Chem.Draw import IPythonConsole
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from rdkit.Chem import Descriptors
from mordred import Calculator, descriptors
from rdkit import Chem
from rdkit.Chem import ChemicalFeatures
from rdkit import RDConfig

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Descriptor 함수 정의

In [5]:
#1
def getMolDescriptors(mol, missingVal=None):
    res = {}
    for nm,fn in Descriptors._descList:
        # some of the descriptor fucntions can throw errors if they fail, catch those here:
        try:
            val = fn(mol)
        except:
            # print the error message:
            import traceback
            traceback.print_exc()
            # and set the descriptor value to whatever missingVal is
            val = missingVal
        res[nm] = val
    return res

#2
def extract_fragments(smiles, hybridization_type=None, exclude_ring=False):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    fragments = []
    for atom in mol.GetAtoms():
        hybridization = atom.GetHybridization()
        if (hybridization_type is None or hybridization == hybridization_type) and (not exclude_ring or not atom.IsInRing()):
            neighbors = atom.GetNeighbors()
            fragment = Chem.EditableMol(Chem.Mol())
            atom_idx = fragment.AddAtom(atom)
            for neighbor in neighbors:
                bond = mol.GetBondBetweenAtoms(atom.GetIdx(), neighbor.GetIdx())
                if bond:
                    bond_order = bond.GetBondType()
                    neighbor_idx = fragment.AddAtom(neighbor)
                    fragment.AddBond(atom_idx, neighbor_idx, bond_order)
            fragments.append(fragment.GetMol())

    return fragments

def count_fragments(full, column_prefix, hybridization_type=None, exclude_ring=False):
    fragment_counts = {}
    for index, row in full.iterrows():
        smiles = row['SMILES']
        fragments = extract_fragments(smiles, hybridization_type, exclude_ring)

        fragment_count = {}
        for idx, fragment in enumerate(fragments):
            fragment_smiles = Chem.MolToSmiles(fragment)
            if fragment_smiles in fragment_count:
                fragment_count[fragment_smiles] += 1
            else:
                fragment_count[fragment_smiles] = 1

        fragment_counts[index] = fragment_count

    result_data = []
    for index, counts in fragment_counts.items():
        counts['SMILES'] = full.loc[index, 'SMILES']
        result_data.append(counts)

    result_df = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')
    result_df.columns = [f'{column_prefix}_{column}' for column in result_df.columns]

    return result_df

#3
def calculate_molecular_properties(full):
    def calculate_stereocenters(smiles):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        chiral_centers = Chem.FindMolChiralCenters(mol, includeUnassigned=True)
        return len(chiral_centers)

    def calculate_aromatic_proportion(smiles):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        aromatic_atoms = [atom for atom in mol.GetAtoms() if atom.GetIsAromatic()]
        total_atoms = mol.GetNumAtoms()
        if total_atoms == 0:
            return None
        return len(aromatic_atoms) / total_atoms

    def calculate_formal_charge(smiles):
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        return Chem.rdmolops.GetFormalCharge(mol)

    properties_data = []

    for index, row in full.iterrows():
        smiles = row['SMILES']
        stereocenters = calculate_stereocenters(smiles)
        proportion = calculate_aromatic_proportion(smiles)
        charge = calculate_formal_charge(smiles)

        if stereocenters is not None and proportion is not None and charge is not None:
            properties_data.append({
                "stereocenters": stereocenters,
                "aromatic_proportion": proportion,
                "formal_charge": charge
            })

    properties_df = pd.DataFrame(properties_data)

    return properties_df

#데이터 가공

In [74]:
train = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/train.csv")
train["AlogP"] = np.where(pd.isna(train["AlogP"]), train["LogD"], train["AlogP"])

#ChEMBL data
MLM = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/train_aug_MLM_0906.csv")
HLM = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/train_aug_HLM_0906.csv")

test = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/test.csv")
test["AlogP"] = np.where(pd.isna(test["AlogP"]), test["LogD"], test["AlogP"])

full = pd.concat([train, test, MLM, HLM], axis = 0).reset_index(drop=True)
full['Molecule'] = full['SMILES'].apply(Chem.MolFromSmiles)

In [124]:
duplicates = full[full.duplicated(subset='SMILES', keep=False)].sort_values(by='SMILES').reset_index(drop=True)
prop = calculate_molecular_properties(duplicates).reset_index(drop=True)
dupli_df = pd.concat([duplicates, prop], axis=1)[['SMILES', 'MLM', 'HLM', 'stereocenters']]

# 'stereocenters' 컬럼 값이 0인 데이터 필터링
filtered_df = dupli_df[dupli_df['stereocenters'] == 0]

# 'SMILES'를 기준으로 그룹화하고 'MLM'과 'HLM'의 평균 계산
mean_mlm = filtered_df.groupby('SMILES')['MLM'].mean().reset_index()
mean_hlm = filtered_df.groupby('SMILES')['HLM'].mean().reset_index()

# 원본 DataFrame을 수정
for index, row in mean_mlm.iterrows():
    smiles = row['SMILES']
    mlm_mean = row['MLM']
    hlm_mean = mean_hlm[mean_hlm['SMILES'] == smiles]['HLM'].values[0]
    dupli_df.loc[dupli_df['SMILES'] == smiles, 'MLM'] = mlm_mean
    dupli_df.loc[dupli_df['SMILES'] == smiles, 'HLM'] = hlm_mean

mean_duplicate = dupli_df.drop_duplicates().reset_index(drop=True)

# 입체이성질체 없는 경우 동일 분자의 실험값들을 mean으로 처리.
for index, row in mean_duplicate.iterrows():
    s_value = row['SMILES']
    matching_indices = full[full['SMILES'] == s_value].index
    for full_index in matching_indices:
        full.at[full_index, 'MLM'] = row['MLM']
        full.at[full_index, 'HLM'] = row['HLM']

filtered_full = full.drop_duplicates(subset=['SMILES', 'HLM', 'MLM'], keep='first').reset_index(drop=True)

In [80]:
full

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,Molecule
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.25900,117.37,<rdkit.Chem.rdchem.Mol object at 0x7be41781e960>
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.17200,73.47,<rdkit.Chem.rdchem.Mol object at 0x7be41781e9d0>
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.58500,62.45,<rdkit.Chem.rdchem.Mol object at 0x7be41781ea40>
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.47500,92.60,<rdkit.Chem.rdchem.Mol object at 0x7be41781eab0>
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.33700,42.43,<rdkit.Chem.rdchem.Mol object at 0x7be41781eb20>
...,...,...,...,...,...,...,...,...,...,...,...,...
6245,CHEMBL3286701,Cc1[nH]c2ccccc2c1CCNC(=O)c1ccc(N(C)C)cc1,0.000,17.000,3.510,321.420,2,2,5,3.51482,NaN,<rdkit.Chem.rdchem.Mol object at 0x7be4180d6dc0>
6246,CHEMBL215387,O=C(O)CCCCCCCCCCCNC(=O)NC12CC3CC(CC(C3)C1)C2,NaN,83.500,5.240,392.580,2,3,13,5.24000,NaN,<rdkit.Chem.rdchem.Mol object at 0x7be4180d6e30>
6247,CHEMBL242459,O=C(NC12CC3CC(CC(C3)C1)C2)N[C@H]1CC[C@H](Oc2cc...,NaN,83.000,4.340,412.530,3,3,5,4.34280,NaN,<rdkit.Chem.rdchem.Mol object at 0x7be4180d6ea0>
6248,CHEMBL3735279,CCOC(=O)c1nc(C)c2c(c1N)C(=O)N(Cc1ccccc1)C2=O,NaN,14.000,1.950,339.350,6,1,4,1.94512,NaN,<rdkit.Chem.rdchem.Mol object at 0x7be4180d6f10>


In [125]:
filtered_full

,id,SMILES,MLM,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,Molecule
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,26.010,50.680,3.259,400.495,5,2,8,3.25900,117.37,<rdkit.Chem.rdchem.Mol object at 0x7be41781e960>
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,29.270,50.590,2.169,301.407,2,1,2,2.17200,73.47,<rdkit.Chem.rdchem.Mol object at 0x7be41781e9d0>
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,5.586,80.892,1.593,297.358,5,0,3,1.58500,62.45,<rdkit.Chem.rdchem.Mol object at 0x7be41781ea40>
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,5.710,2.000,4.771,494.652,6,0,5,3.47500,92.60,<rdkit.Chem.rdchem.Mol object at 0x7be41781eab0>
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,93.270,99.990,2.335,268.310,3,0,1,2.33700,42.43,<rdkit.Chem.rdchem.Mol object at 0x7be41781eb20>
...,...,...,...,...,...,...,...,...,...,...,...,...
5742,CHEMBL3810411,Cc1cc(CCC#N)cc(C)c1Oc1cc(Nc2ccc(C#N)cc2)c(N)cc...,NaN,78.400,6.000,451.570,6,3,9,6.00140,NaN,<rdkit.Chem.rdchem.Mol object at 0x7be4180d6c00>
5743,CHEMBL3092907,Cc1nc(N2CCN(C)CC2)c2nc(-c3ccccc3Cl)n(CCO)c2n1.Cl,NaN,13.000,2.200,423.350,7,1,4,2.62102,NaN,<rdkit.Chem.rdchem.Mol object at 0x7be4180d6c70>
5744,CHEMBL2177757,O=C(Cc1ccccc1)Nc1nnc(CCSCCc2nnc(NC(=O)Cc3ccccc...,NaN,56.000,4.270,524.700,9,2,12,4.27040,NaN,<rdkit.Chem.rdchem.Mol object at 0x7be4180d6ce0>
5745,CHEMBL3735279,CCOC(=O)c1nc(C)c2c(c1N)C(=O)N(Cc1ccccc1)C2=O,NaN,14.000,1.950,339.350,6,1,4,1.94512,NaN,<rdkit.Chem.rdchem.Mol object at 0x7be4180d6f10>


In [82]:
# Mordred
calc = Calculator(descriptors, ignore_3D=True)
mord_desc_df = calc.pandas([Chem.MolFromSmiles(x) for x in filtered_full.SMILES])
mord_desc_df.columns = ['mord_'+ column for column in mord_desc_df.columns]
# object 형식 열 삭제
mord_desc_df.drop(columns=mord_desc_df.select_dtypes(include=['object']).columns, inplace=True)

# bool 형식 열을 0과 1로 인코딩
bool_columns = mord_desc_df.select_dtypes(include=['bool']).columns
mord_desc_df[bool_columns] = mord_desc_df[bool_columns].astype(int)

print(mord_desc_df.head())

  4%|▍         | 249/5747 [04:41<3:56:28,  2.58s/it]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


 95%|█████████▍| 5434/5747 [1:46:18<06:09,  1.18s/it]

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


100%|██████████| 5747/5747 [1:52:50<00:00,  1.18s/it]


    mord_ABC  mord_ABCGG  mord_nAcid  mord_nBase  mord_nAromAtom  \
0  21.379612   17.449011           0           0              16   
1  16.539255   14.049653           0           0              11   
2  17.475469   13.660693           2           1              13   
3  27.857311   20.034364           0           1              17   
4  15.722758   12.817176           0           0              12   

   mord_nAromBond  mord_nAtom  mord_nHeavyAtom  mord_nSpiro  mord_nBridgehead  \
0              16          52               28            0                 0   
1              11          40               21            0                 0   
2              15          41               22            0                 0   
3              17          69               35            0                 0   
4              12          36               20            0                 0   

   ...  mord_SRW09  mord_SRW10  mord_TSRW10     mord_MW  mord_AMW  mord_WPath  \
0  ...    7.390799   10

In [83]:
from rdkit import DataStructs

allDescrs = [getMolDescriptors(m) for m in filtered_full['Molecule']]
full_Descrs = pd.DataFrame(allDescrs)

In [117]:
result_df_sp3 = count_fragments(filtered_full, 'sp3', Chem.HybridizationType.SP3)
result_df_non_ring_sp3 = count_fragments(filtered_full, 'sp3', Chem.HybridizationType.SP3, exclude_ring=True)
result_df_sp2 = count_fragments(filtered_full, 'sp2', Chem.HybridizationType.SP2)
result_df_non_ring_sp2 = count_fragments(filtered_full, 'non_ring_sp2', Chem.HybridizationType.SP2, exclude_ring=True)
result_df_sp = count_fragments(filtered_full, 'sp', Chem.HybridizationType.SP)

prop_df = calculate_molecular_properties(filtered_full)

In [118]:
result_df_non_ring_sp3

,sp3_CC,sp3_CCO,sp3_cCN,sp3_Cc,sp3_CCC,sp3_CCN,sp3_cS(N)(=O)=O,sp3_CO,sp3_cF,sp3_CN,...,sp3_NP(=O)(O)O,sp3_CC(C)[N+],sp3_C[C@@H](C)S,sp3_O=P([O-])(O)O,sp3_[O-]P,sp3_OP,sp3_C[Si],sp3_CC(P)P,sp3_c[Si](C)(C)C,sp3_c[C@@H](C)O
0,2.0,2.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5742,0.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5743,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5744,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5745,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [111]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import ChemicalFeatures
from rdkit import RDConfig
import os

# Define the feature definition file
fdefName = os.path.join(RDConfig.RDDataDir, 'BaseFeatures.fdef')

# Build the feature factory
factory = ChemicalFeatures.BuildFeatureFactory(fdefName)

# Initialize a dictionary to store feature counts
feature_counts = {}

def extract_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None

    # Get the features for the molecule
    feats = factory.GetFeaturesForMol(mol)

    feature_count = {}
    for feat in feats:
        # feat_type = feat.GetFamily()
        feat_type = feat.GetType()  # Use the feature type (or type)
        if feat_type in feature_count:
            feature_count[feat_type] += 1
        else:
            feature_count[feat_type] = 1

    return feature_count

# Iterate through each SMILES in the DataFrame
for index, row in filtered_full.iterrows():
    smiles = row['SMILES']
    feature_count = extract_features(smiles)
    feature_counts[index] = feature_count

# Create a DataFrame from the feature counts dictionary and fill missing values with 0
result_data = []
for index, counts in feature_counts.items():
    counts['SMILES'] = filtered_full.loc[index, 'SMILES']
    result_data.append(counts)

result_df_features = pd.DataFrame(result_data).fillna(0).drop(columns='SMILES')

# Rename columns with a prefix
result_df_features.columns = ['feature_' + column for column in result_df_features.columns]

# Print the resulting DataFrame
print(result_df_features)


      feature_SingleAtomDonor  feature_SingleAtomAcceptor  feature_Arom5  \
0                         3.0                         5.0            2.0   
1                         1.0                         2.0            1.0   
2                         1.0                         4.0            1.0   
3                         1.0                         5.0            1.0   
4                         0.0                         3.0            0.0   
...                       ...                         ...            ...   
5742                      3.0                         1.0            0.0   
5743                      2.0                         4.0            1.0   
5744                      2.0                         6.0            2.0   
5745                      2.0                         5.0            0.0   
5746                      3.0                         4.0            1.0   

      feature_Arom6  feature_ThreeWayAttach  feature_RH6_6  \
0               1.0      

In [126]:
Data = pd.concat([filtered_full, full_Descrs,mord_desc_df,result_df_sp3, result_df_non_ring_sp3, result_df_sp2, result_df_non_ring_sp2, result_df_sp, prop_df, result_df_features], axis=1).drop(columns=['Molecule', 'MolWt', 'NumHAcceptors', 'NumHDonors', 'NumRotatableBonds', 'MolLogP'])
Data["Molecular_PolarSurfaceArea"] = np.where(pd.isna(Data["Molecular_PolarSurfaceArea"]), Data["TPSA"], Data["Molecular_PolarSurfaceArea"])

In [127]:
Data['beyond_Lipinski'] = 0

Data.loc[Data['Molecular_Weight'] >= 500, 'beyond_Lipinski'] += 1
Data.loc[Data['LogD'] >= 5, 'beyond_Lipinski'] += 1
Data.loc[Data['Num_H_Donors'] >= 5, 'beyond_Lipinski'] += 1
Data.loc[Data['Num_H_Acceptors'] >= 10, 'beyond_Lipinski'] += 1
Data.loc[Data['Num_RotatableBonds'] >= 10, 'beyond_Lipinski'] += 1
Data.loc[(Data['Num_H_Donors'] + Data['Num_H_Acceptors']) >= 12, 'beyond_Lipinski'] += 1
Data.loc[Data['TPSA'] >= 140, 'beyond_Lipinski'] += 1
Data.loc[Data['FractionCSP3'] <= 0.3, 'beyond_Lipinski'] += 1
Data.loc[Data['NumAromaticRings'] >= 5, 'beyond_Lipinski'] += 1
Data.loc[Data['NOCount'] >= 10, 'beyond_Lipinski'] += 1
Data.loc[Data['NHOHCount'] >= 5, 'beyond_Lipinski'] += 1

In [128]:
# Specify the file path where you want to save the pickle file
pickle_file_path = "/content/drive/MyDrive/dacon_data_0911.pkl"

# Save the DataFrame as a pickle file
Data.to_pickle(pickle_file_path)

# Print a message to confirm that the file has been saved
print(f"DataFrame saved as {pickle_file_path}")

DataFrame saved as /content/drive/MyDrive/dacon_data_0911.pkl


In [6]:
import pandas as pd

# 지정된 pickle 파일 경로
pickle_file_path = "/content/drive/MyDrive/dacon_data_0911.pkl"

# pickle 파일을 DataFrame으로 로드
Data = pd.read_pickle(pickle_file_path)

# 로드된 DataFrame 출력 또는 사용
print(Data)

                 id                                             SMILES  \
0        TRAIN_0000    CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC   
1        TRAIN_0001               Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1   
2        TRAIN_0002                   CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1   
3        TRAIN_0003  Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...   
4        TRAIN_0004                Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2   
...             ...                                                ...   
5742  CHEMBL3810411  Cc1cc(CCC#N)cc(C)c1Oc1cc(Nc2ccc(C#N)cc2)c(N)cc...   
5743  CHEMBL3092907   Cc1nc(N2CCN(C)CC2)c2nc(-c3ccccc3Cl)n(CCO)c2n1.Cl   
5744  CHEMBL2177757  O=C(Cc1ccccc1)Nc1nnc(CCSCCc2nnc(NC(=O)Cc3ccccc...   
5745  CHEMBL3735279       CCOC(=O)c1nc(C)c2c(c1N)C(=O)N(Cc1ccccc1)C2=O   
5746  CHEMBL3353541  C[C@]1(C(=O)N(CCCC(=O)O)Cc2cccc(Cl)c2)CCN1C(=O...   

         MLM     HLM  AlogP  Molecular_Weight  Num_H_Acceptors  Num_H_Donors  \
0     26.010  50.680  3.259    

In [32]:
data_column = [column for column in Data.columns]

In [ ]:
Desc_feature= ['MaxAbsEStateIndex','MaxEStateIndex','MinAbsEStateIndex','MinEStateIndex','qed','HeavyAtomMolWt','NumValenceElectrons',
 'NumRadicalElectrons','MaxPartialCharge','MinPartialCharge','MaxAbsPartialCharge','MinAbsPartialCharge',
 'FpDensityMorgan1','FpDensityMorgan2','FpDensityMorgan3','HallKierAlpha','Ipc','NHOHCount','NOCount']

In [7]:
# 'id' 컬럼에 'TEST' 문자열이 들어가는 행을 testset으로 선택
test = Data[Data['id'].str.contains('TEST')]
print(len(test))
# 'id' 컬럼에 'TEST' 문자열이 들어가지 않는 행을 trainset으로 선택
train = Data[~Data['id'].str.contains('TEST')]
print(len(train))

483
5264


In [8]:
from sklearn.preprocessing import StandardScaler, RobustScaler

# 숫자형식 컬럼들의 min-max 정규화
scaler = MinMaxScaler()
standard_scaler = StandardScaler()
robust_scaler = RobustScaler()
non_features = ['id', 'SMILES', 'MLM', 'HLM','Fingerprint']
features = [column for column in train.columns if column not in non_features]
train[features] = scaler.fit_transform(train[features])

test[features] = scaler.transform(test[features])

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasatt

In [9]:
train_HLM = train.drop(columns=['MLM']).dropna(axis=0)
print(len(train_HLM))
train_MLM = train.drop(columns=['HLM']).dropna(axis=0)
print(len(train_MLM))

4842
4045


In [10]:
train_HLM

,id,SMILES,HLM,AlogP,Molecular_Weight,Num_H_Acceptors,Num_H_Donors,Num_RotatableBonds,LogD,Molecular_PolarSurfaceArea,...,feature_RH4_4,feature_Guanidine,feature_tButyl,feature_PosN,feature_RH3_3,feature_Nitro2,feature_ZnBinder6,feature_Arom7,feature_ZnBinder4,beyond_Lipinski
0,TRAIN_0000,CCOc1ccc(CNC(=O)c2cc(-c3sc(C)nc3C)n[nH]2)cc1OCC,50.680,0.554964,0.192138,0.294118,0.1250,0.210526,0.579660,0.278787,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,TRAIN_0001,Cc1nc(C)c(CN2CC(C)C(=O)Nc3ccccc32)s1,50.590,0.473175,0.108751,0.117647,0.0625,0.052632,0.502622,0.171552,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,TRAIN_0002,CCCN1CCN(c2nn3nnnc3c3ccccc23)CC1,80.892,0.429954,0.105343,0.294118,0.0000,0.078947,0.461021,0.144633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,TRAIN_0003,Cc1ccc(-c2ccc(-n3nc(C)c(S(=O)(=O)N4CCN(C5CCCCC...,2.000,0.668417,0.271376,0.352941,0.0000,0.131579,0.594968,0.218281,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,TRAIN_0004,Cc1ccc2c(c1)N(C(=O)c1ccncc1)CC(C)O2,99.990,0.485631,0.080898,0.176471,0.0000,0.026316,0.514316,0.095730,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5742,CHEMBL3810411,Cc1cc(CCC#N)cc(C)c1Oc1cc(Nc2ccc(C#N)cc2)c(N)cc...,78.400,0.760636,0.235120,0.352941,0.1875,0.236842,0.774018,0.253188,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2
5743,CHEMBL3092907,Cc1nc(N2CCN(C)CC2)c2nc(-c3ccccc3Cl)n(CCO)c2n1.Cl,13.000,0.475501,0.211371,0.411765,0.0625,0.105263,0.534445,0.163833,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5744,CHEMBL2177757,O=C(Cc1ccccc1)Nc1nnc(CCSCCc2nnc(NC(=O)Cc3ccccc...,56.000,0.630825,0.296662,0.529412,0.1250,0.315789,0.651339,0.260198,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3
5745,CHEMBL3735279,CCOC(=O)c1nc(C)c2c(c1N)C(=O)N(Cc1ccccc1)C2=O,14.000,0.456742,0.140682,0.352941,0.0625,0.105263,0.486543,0.242684,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1


In [36]:
!pip install dgl

In [24]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import mean_squared_error
import math
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from keras.regularizers import l2

# Define features and targets
non_features = ['id', 'SMILES', 'MLM', 'HLM', 'Fingerprint']
features = [column for column in train.columns if column not in non_features]
mlm_target = "MLM"
hlm_target = "HLM"

# Initialize KFold
seed = 42
n_splits = 20
kf = KFold(n_splits=n_splits, random_state=seed, shuffle=True)

# Initialize arrays to store models and scores
reg_mlms = []
reg_hlms = []

# Initialize arrays to store RMSE scores
mlm_rmse_scores = []
hlm_rmse_scores = []

# Loop through KFold splits
for i, (train_index, valid_index) in enumerate(kf.split(train_HLM)):
    df_train = train_HLM.iloc[train_index]
    df_valid = train_HLM.iloc[valid_index]

    x_train_num = df_train[features].values

    y_hlm_train = df_train[hlm_target].values

    x_valid_num = df_valid[features].values

    y_hlm_valid = df_valid[hlm_target].values


    # 이미지 입력 경로
    x1_input = keras.Input(shape=(x_train_num.shape[1],))
    x1 = layers.Dense(1024, activation='relu')(x1_input)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Dropout(0.2)(x1)
    merged = layers.Dense(1024, activation='relu')(x1)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(512, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(256, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(128, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)

    # 회귀 모델 출력 레이어 추가
    outputs = layers.Dense(1, kernel_regularizer='l1_l2')(merged)

    # 모델 생성
    model_hlm = keras.Model(inputs=x1_input, outputs=outputs)

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=8,
    decay_rate=0.98)
    optimizer_decay = keras.optimizers.SGD(learning_rate=lr_schedule)

    model_hlm.compile(optimizer=optimizer_decay, loss='mean_squared_error', metrics=[RootMeanSquaredError()])

    checkpoint_hlm = ModelCheckpoint(f'model_hlm_fold{i}.h5', monitor='val_loss', verbose=0, save_best_only=True)
    early_stopping_hlm = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    model_hlm.fit(x_train_num, y_hlm_train, epochs=500, batch_size=16, verbose=1, validation_data=(x_valid_num, y_hlm_valid),
                  callbacks=[checkpoint_hlm, early_stopping_hlm])

    reg_hlms.append(model_hlm)

    # Calculate RMSE for HLM predictions
    y_hlm_pred = model_hlm.predict(x_valid_num)
    hlm_rmse = math.sqrt(mean_squared_error(y_hlm_valid, y_hlm_pred))
    print(hlm_rmse)
    hlm_rmse_scores.append(hlm_rmse)

Epoch 1/500
288/288 [==============================] - 4s 7ms/step - loss: 1291.3960 - root_mean_squared_error: 35.9225 - val_loss: 912.0453 - val_root_mean_squared_error: 30.1837
Epoch 2/500
 10/288 [>.............................] - ETA: 1s - loss: 981.8726 - root_mean_squared_error: 31.3187 

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


288/288 [==============================] - 2s 6ms/step - loss: 966.7283 - root_mean_squared_error: 31.0761 - val_loss: 898.9067 - val_root_mean_squared_error: 29.9655
Epoch 3/500
288/288 [==============================] - 2s 6ms/step - loss: 894.0826 - root_mean_squared_error: 29.8845 - val_loss: 826.2173 - val_root_mean_squared_error: 28.7263
Epoch 4/500
288/288 [==============================] - 2s 7ms/step - loss: 870.4010 - root_mean_squared_error: 29.4855 - val_loss: 808.1194 - val_root_mean_squared_error: 28.4094
Epoch 5/500
288/288 [==============================] - 2s 7ms/step - loss: 836.1818 - root_mean_squared_error: 28.8993 - val_loss: 804.5114 - val_root_mean_squared_error: 28.3460
Epoch 6/500
288/288 [==============================] - 2s 7ms/step - loss: 812.5217 - root_mean_squared_error: 28.4870 - val_loss: 800.1597 - val_root_mean_squared_error: 28.2691
Epoch 7/500
288/288 [==============================] - 2s 7ms/step - loss: 801.3748 - root_mean_squared_error: 28.290

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


288/288 [==============================] - 2s 7ms/step - loss: 952.9602 - root_mean_squared_error: 30.8540 - val_loss: 989.0490 - val_root_mean_squared_error: 31.4335
Epoch 3/500
288/288 [==============================] - 2s 7ms/step - loss: 905.3743 - root_mean_squared_error: 30.0730 - val_loss: 895.5146 - val_root_mean_squared_error: 29.9084
Epoch 4/500
288/288 [==============================] - 2s 7ms/step - loss: 852.3433 - root_mean_squared_error: 29.1779 - val_loss: 894.2679 - val_root_mean_squared_error: 29.8875
Epoch 5/500
288/288 [==============================] - 2s 6ms/step - loss: 818.9998 - root_mean_squared_error: 28.6008 - val_loss: 899.9042 - val_root_mean_squared_error: 29.9816
Epoch 6/500
288/288 [==============================] - 2s 6ms/step - loss: 813.3651 - root_mean_squared_error: 28.5019 - val_loss: 897.2601 - val_root_mean_squared_error: 29.9376
Epoch 7/500
288/288 [==============================] - 2s 6ms/step - loss: 808.9669 - root_mean_squared_error: 28.424

...

288/288 [==============================] - 2s 6ms/step - loss: 961.7115 - root_mean_squared_error: 30.9950 - val_loss: 928.8490 - val_root_mean_squared_error: 30.4607
Epoch 3/500
288/288 [==============================] - 2s 6ms/step - loss: 878.0258 - root_mean_squared_error: 29.6143 - val_loss: 924.3303 - val_root_mean_squared_error: 30.3858
Epoch 4/500
288/288 [==============================] - 2s 6ms/step - loss: 849.0795 - root_mean_squared_error: 29.1213 - val_loss: 942.8671 - val_root_mean_squared_error: 30.6896
Epoch 5/500
288/288 [==============================] - 2s 6ms/step - loss: 834.6227 - root_mean_squared_error: 28.8721 - val_loss: 922.4080 - val_root_mean_squared_error: 30.3543
Epoch 6/500
288/288 [==============================] - 2s 6ms/step - loss: 805.4034 - root_mean_squared_error: 28.3615 - val_loss: 926.3940 - val_root_mean_squared_error: 30.4198
Epoch 7/500
288/288 [==============================] - 2s 7ms/step - loss: 789.4323 - root_mean_squared_error: 28.078

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


288/288 [==============================] - 2s 7ms/step - loss: 962.1399 - root_mean_squared_error: 31.0026 - val_loss: 929.1572 - val_root_mean_squared_error: 30.4657
Epoch 3/500
288/288 [==============================] - 2s 6ms/step - loss: 902.8282 - root_mean_squared_error: 30.0307 - val_loss: 1011.1262 - val_root_mean_squared_error: 31.7829
Epoch 4/500
288/288 [==============================] - 2s 6ms/step - loss: 851.2714 - root_mean_squared_error: 29.1596 - val_loss: 913.1437 - val_root_mean_squared_error: 30.2019
Epoch 5/500
288/288 [==============================] - 2s 6ms/step - loss: 807.4139 - root_mean_squared_error: 28.3975 - val_loss: 910.5735 - val_root_mean_squared_error: 30.1593
Epoch 6/500
288/288 [==============================] - 2s 6ms/step - loss: 810.3943 - root_mean_squared_error: 28.4500 - val_loss: 901.5732 - val_root_mean_squared_error: 30.0096
Epoch 7/500
288/288 [==============================] - 2s 6ms/step - loss: 802.2261 - root_mean_squared_error: 28.30

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


288/288 [==============================] - 2s 6ms/step - loss: 967.2577 - root_mean_squared_error: 31.0845 - val_loss: 999.2137 - val_root_mean_squared_error: 31.5949
Epoch 3/500
288/288 [==============================] - 2s 6ms/step - loss: 901.8840 - root_mean_squared_error: 30.0146 - val_loss: 912.6188 - val_root_mean_squared_error: 30.1927
Epoch 4/500
288/288 [==============================] - 2s 7ms/step - loss: 858.3898 - root_mean_squared_error: 29.2810 - val_loss: 888.4388 - val_root_mean_squared_error: 29.7896
Epoch 5/500
288/288 [==============================] - 2s 6ms/step - loss: 835.7283 - root_mean_squared_error: 28.8914 - val_loss: 908.7468 - val_root_mean_squared_error: 30.1286
Epoch 6/500
288/288 [==============================] - 2s 6ms/step - loss: 833.2374 - root_mean_squared_error: 28.8482 - val_loss: 901.3223 - val_root_mean_squared_error: 30.0051
Epoch 7/500
288/288 [==============================] - 2s 6ms/step - loss: 792.6755 - root_mean_squared_error: 28.136

In [25]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras.optimizers import Adam
from keras.metrics import RootMeanSquaredError
import keras
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import mean_squared_error
import math
from tensorflow.keras.optimizers.schedules import ExponentialDecay

# Define features and targets
non_features = ['id', 'SMILES', 'MLM', 'HLM', 'Fingerprint']
features = [column for column in train.columns if column not in non_features]
mlm_target = "MLM"
hlm_target = "HLM"

# Initialize KFold
seed = 42
n_splits = 20
kf = KFold(n_splits=n_splits, random_state=seed, shuffle=True)

# Initialize arrays to store models and scores
reg_mlms = []
reg_hlms = []

# Initialize arrays to store RMSE scores
mlm_rmse_scores = []
hlm_rmse_scores = []

# Loop through KFold splits
for i, (train_index, valid_index) in enumerate(kf.split(train_MLM)):
    df_train = train_MLM.iloc[train_index]
    df_valid = train_MLM.iloc[valid_index]

    x_train_num = df_train[features].values

    y_mlm_train = df_train[mlm_target].values

    x_valid_num = df_valid[features].values

    y_mlm_valid = df_valid[mlm_target].values

    # 이미지 입력 경로
    x1_input = keras.Input(shape=(x_train_num.shape[1],))
    x1 = layers.Dense(1024, activation='relu')(x1_input)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Dropout(0.2)(x1)
    merged = layers.Dense(1024, activation='relu')(x1)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(512, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(256, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(128, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)

    # 회귀 모델 출력 레이어 추가
    outputs = layers.Dense(1)(merged)

    # 모델 생성
    model = keras.Model(inputs=x1_input, outputs=outputs)

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=8,
    decay_rate=0.98)
    optimizer_decay = keras.optimizers.SGD(learning_rate=lr_schedule)

    model.compile(optimizer=optimizer_decay, loss='mean_squared_error', metrics=[RootMeanSquaredError()])

    # Create and compile another model for HLM
    # 이미지 입력 경로
    x1_input = keras.Input(shape=(x_train_num.shape[1],))
    x1 = layers.Dense(256, activation='relu')(x1_input)
    x1 = layers.BatchNormalization()(x1)
    x1 = layers.Dropout(0.2)(x1)
    merged = layers.Dense(1024, activation='relu')(x1)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(512, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(256, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)
    merged = layers.Dropout(0.2)(merged)
    merged = layers.Dense(128, activation='relu')(merged)
    merged = layers.BatchNormalization()(merged)

    # 회귀 모델 출력 레이어 추가
    outputs = layers.Dense(1)(merged)

    # 모델 생성
    model_hlm = keras.Model(inputs=x1_input, outputs=outputs)

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.001,
    decay_steps=8,
    decay_rate=0.98)
    optimizer_decay = keras.optimizers.SGD(learning_rate=lr_schedule)

    model_hlm.compile(optimizer=optimizer_decay, loss='mean_squared_error', metrics=[RootMeanSquaredError()])


    # Train the model
    checkpoint_mlm = ModelCheckpoint(f'model_mlm_fold{i}.h5', monitor='val_loss', verbose=0, save_best_only=True)
    early_stopping_mlm = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    model.fit(x_train_num, y_mlm_train, epochs=500, batch_size=16, verbose=1, validation_data=(x_valid_num, y_mlm_valid),
              callbacks=[checkpoint_mlm, early_stopping_mlm])

    reg_mlms.append(model)

    # checkpoint_hlm = ModelCheckpoint(f'model_hlm_fold{i}.h5', monitor='val_loss', verbose=0, save_best_only=True)
    # early_stopping_hlm = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

    # model_hlm.fit(x_train_num, y_hlm_train, epochs=500, batch_size=16, verbose=1, validation_data=(x_valid_num, y_hlm_valid),
    #               callbacks=[checkpoint_hlm, early_stopping_hlm])

    # reg_hlms.append(model_hlm)

    # Calculate RMSE for MLM predictions
    y_mlm_pred = model.predict(x_valid_num)
    mlm_rmse = math.sqrt(mean_squared_error(y_mlm_valid, y_mlm_pred))
    print(mlm_rmse)
    mlm_rmse_scores.append(mlm_rmse)

    # # Calculate RMSE for HLM predictions
    # y_hlm_pred = model_hlm.predict(x_valid_num)
    # hlm_rmse = math.sqrt(mean_squared_error(y_hlm_valid, y_hlm_pred))
    # print(hlm_rmse)
    # hlm_rmse_scores.append(hlm_rmse)

Epoch 1/500
241/241 [==============================] - 4s 8ms/step - loss: 1222.0164 - root_mean_squared_error: 34.9573 - val_loss: 3813.9473 - val_root_mean_squared_error: 61.7572
Epoch 2/500
 10/241 [>.............................] - ETA: 1s - loss: 1000.3320 - root_mean_squared_error: 31.6280

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


241/241 [==============================] - 2s 7ms/step - loss: 933.0826 - root_mean_squared_error: 30.5464 - val_loss: 994.5351 - val_root_mean_squared_error: 31.5363
Epoch 3/500
241/241 [==============================] - 1s 6ms/step - loss: 877.6008 - root_mean_squared_error: 29.6243 - val_loss: 1053.8179 - val_root_mean_squared_error: 32.4626
Epoch 4/500
241/241 [==============================] - 1s 6ms/step - loss: 836.5040 - root_mean_squared_error: 28.9224 - val_loss: 1001.8346 - val_root_mean_squared_error: 31.6518
Epoch 5/500
241/241 [==============================] - 2s 7ms/step - loss: 793.2324 - root_mean_squared_error: 28.1644 - val_loss: 984.6315 - val_root_mean_squared_error: 31.3788
Epoch 6/500
241/241 [==============================] - 2s 7ms/step - loss: 775.9859 - root_mean_squared_error: 27.8565 - val_loss: 967.1755 - val_root_mean_squared_error: 31.0994
Epoch 7/500
241/241 [==============================] - 2s 7ms/step - loss: 771.7156 - root_mean_squared_error: 27.7

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


241/241 [==============================] - 2s 7ms/step - loss: 942.3912 - root_mean_squared_error: 30.6984 - val_loss: 998.3001 - val_root_mean_squared_error: 31.5959
Epoch 3/500
241/241 [==============================] - 2s 7ms/step - loss: 866.7396 - root_mean_squared_error: 29.4404 - val_loss: 958.6582 - val_root_mean_squared_error: 30.9622
Epoch 4/500
241/241 [==============================] - 2s 7ms/step - loss: 841.3298 - root_mean_squared_error: 29.0057 - val_loss: 872.6892 - val_root_mean_squared_error: 29.5413
Epoch 5/500
241/241 [==============================] - 2s 7ms/step - loss: 801.1785 - root_mean_squared_error: 28.3051 - val_loss: 848.4699 - val_root_mean_squared_error: 29.1285
Epoch 6/500
241/241 [==============================] - 2s 7ms/step - loss: 785.6418 - root_mean_squared_error: 28.0293 - val_loss: 846.9062 - val_root_mean_squared_error: 29.1017
Epoch 7/500
241/241 [==============================] - 2s 7ms/step - loss: 774.7936 - root_mean_squared_error: 27.835

...

241/241 [==============================] - 2s 6ms/step - loss: 932.6884 - root_mean_squared_error: 30.5399 - val_loss: 919.3033 - val_root_mean_squared_error: 30.3200
Epoch 3/500
241/241 [==============================] - 1s 6ms/step - loss: 858.8267 - root_mean_squared_error: 29.3057 - val_loss: 981.5154 - val_root_mean_squared_error: 31.3291
Epoch 4/500
241/241 [==============================] - 2s 7ms/step - loss: 824.6358 - root_mean_squared_error: 28.7165 - val_loss: 897.2676 - val_root_mean_squared_error: 29.9544
Epoch 5/500
241/241 [==============================] - 2s 7ms/step - loss: 813.5470 - root_mean_squared_error: 28.5227 - val_loss: 862.2141 - val_root_mean_squared_error: 29.3635
Epoch 6/500
241/241 [==============================] - 1s 6ms/step - loss: 773.5362 - root_mean_squared_error: 27.8125 - val_loss: 875.4138 - val_root_mean_squared_error: 29.5874
Epoch 7/500
241/241 [==============================] - 2s 6ms/step - loss: 768.8917 - root_mean_squared_error: 27.728

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


241/241 [==============================] - 1s 6ms/step - loss: 936.8481 - root_mean_squared_error: 30.6080 - val_loss: 104165.6641 - val_root_mean_squared_error: 322.7471
Epoch 3/500
241/241 [==============================] - 1s 6ms/step - loss: 943.4705 - root_mean_squared_error: 30.7160 - val_loss: 1071.3113 - val_root_mean_squared_error: 32.7309
Epoch 4/500
241/241 [==============================] - 2s 6ms/step - loss: 851.7455 - root_mean_squared_error: 29.1847 - val_loss: 931.8867 - val_root_mean_squared_error: 30.5268
Epoch 5/500
241/241 [==============================] - 1s 6ms/step - loss: 830.3231 - root_mean_squared_error: 28.8153 - val_loss: 957.3655 - val_root_mean_squared_error: 30.9413
Epoch 6/500
241/241 [==============================] - 1s 6ms/step - loss: 806.7167 - root_mean_squared_error: 28.4028 - val_loss: 952.4528 - val_root_mean_squared_error: 30.8618
Epoch 7/500
241/241 [==============================] - 1s 6ms/step - loss: 776.2912 - root_mean_squared_error: 2

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


241/241 [==============================] - 2s 6ms/step - loss: 945.8702 - root_mean_squared_error: 30.7550 - val_loss: 2056.6865 - val_root_mean_squared_error: 45.3507
Epoch 3/500
241/241 [==============================] - 2s 7ms/step - loss: 880.2855 - root_mean_squared_error: 29.6696 - val_loss: 878.8383 - val_root_mean_squared_error: 29.6452
Epoch 4/500
241/241 [==============================] - 2s 7ms/step - loss: 832.4879 - root_mean_squared_error: 28.8529 - val_loss: 876.9957 - val_root_mean_squared_error: 29.6141
Epoch 5/500
241/241 [==============================] - 2s 7ms/step - loss: 819.2654 - root_mean_squared_error: 28.6228 - val_loss: 857.5165 - val_root_mean_squared_error: 29.2834
Epoch 6/500
241/241 [==============================] - 1s 6ms/step - loss: 775.0018 - root_mean_squared_error: 27.8389 - val_loss: 890.6502 - val_root_mean_squared_error: 29.8438
Epoch 7/500
241/241 [==============================] - 1s 6ms/step - loss: 764.6990 - root_mean_squared_error: 27.65

In [26]:
from keras.models import load_model

# Load MLM models
mlm_models = [load_model(f'model_mlm_fold{i}.h5') for i in range(n_splits)]

# Load HLM models
hlm_models = [load_model(f'model_hlm_fold{i}.h5') for i in range(n_splits)]

x_test = test[features].values

mlm_predictions = []
hlm_predictions = []

for model_mlm, model_hlm in zip(mlm_models, hlm_models):
    mlm_predictions.append(model_mlm.predict(x_test))
    hlm_predictions.append(model_hlm.predict(x_test))

# Convert prediction lists to numpy arrays
mlm_predictions = np.array(mlm_predictions)
hlm_predictions = np.array(hlm_predictions)

mlm_ensemble_prediction = mlm_predictions.mean(axis=0)
hlm_ensemble_prediction = hlm_predictions.mean(axis=0)

16/16 [==============================] - 0s 2ms/step


In [27]:
df_submission = pd.read_csv("/content/drive/MyDrive/metabolism_dacon/sample_submission.csv")
df_submission["MLM"] = mlm_ensemble_prediction
df_submission["HLM"] = hlm_ensemble_prediction
df_submission.to_csv("submission.csv", index = False, encoding = "utf-8-sig")

In [28]:
df_submission

,id,MLM,HLM
0,TEST_000,22.072287,49.431904
1,TEST_001,67.795456,80.603355
2,TEST_002,48.650043,63.674030
3,TEST_003,49.560764,69.864143
4,TEST_004,60.214588,75.319855
...,...,...,...
478,TEST_478,6.380654,25.069542
479,TEST_479,72.472137,86.032578
480,TEST_480,38.472294,64.516289
481,TEST_481,44.892677,71.460587
